### Cross-Validation and the Bootstrap

resampling techniques

In [5]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS,
                         summarize,
                         poly)
from sklearn.model_selection import train_test_split

In [7]:
from functools import partial
from sklearn.model_selection import \
     (cross_validate,
      KFold,
      ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

### Validation set Approach

We use the validation set approach in order to estimate the test error rates thatresult from fitting various linear models on the Auto data set

Here we use test_train_split() function to split the data into training and vaidation sets.

We be using random seed to deal with random elements.

In [13]:
Auto = load_data('Auto')
Auto_train, Auto_valid = train_test_split(Auto,
                                         test_size=196,
                                         random_state=0)

There are 392 obervation in this data set we use train test split to split them equally of size 196 using test_size=196

In [20]:
hp_mm = MS(['horsepower'])
X_train = hp_mm.fit_transform(Auto_train)
y_train = Auto_train['mpg']
model = sm.OLS(y_train, X_train)
results = model.fit()

with Auto_train we will fit a linear regression using only the obervations corresponding to the training set.

In [25]:
X_valid = hp_mm.transform(Auto_valid)
y_valid = Auto_valid['mpg']
valid_pred = results.predict(X_valid)
np.mean((y_valid - valid_pred)**2)

23.61661706966988

There is a method of results i.e predict() which is used to evaluate on the model matrix for this model created using the validation data set. we have also calculated the MSE of our model.

Here validation dataset is used for hyperparameter tuning. for checking overfitting!

In [29]:
def evalMSE(terms,
            response,
            train,
            test):

   mm = MS(terms)
   X_train = mm.fit_transform(train)
   y_train = train[response]

   X_test = mm.transform(test)
   y_test = test[response]

   results = sm.OLS(y_train, X_train).fit()
   test_pred = results.predict(X_test)

   return np.mean((y_test - test_pred)**2)

we provide a function evalMSE() that takes a model string as well as a training and test set and returns the MSE on the test set.

In [32]:
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

we use enumerate() function here to estimate the validation MSE using linear, Quadratic and cubic fits. It will give both the values and indices of objects as one iterates over a for loop.

In [37]:
Auto_train, Auto_valid = train_test_split(Auto,
                                          test_size=196,
                                          random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1, 4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       Auto_train,
                       Auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

using this split of observations into a training set and a validation set, we find that the validation set error rates for the models with linar, quadratic and cubic terms are 20.76, 16.95, 16.97. These results are cosistent with the previous findings thata model that predicts `mpg` using a quadratic function of `horsepower` performs better than a model that involves only a linear function of horsepower and there is no evidence of an improvement in using a cubic fucntion of horsepower.

### Cross-Validation

Cross-validation helps check how well a model works. In Python, it’s easiest to do this using sklearn. But if you build your model using statsmodels, it won’t work directly with sklearn. To fix this, we use a `wrapper` called sklearn_sm() from the ISLP package. This wrapper connects statsmodels models with sklearn tools. You can also give it extra info like the formula and model settings. This makes cross-validation simple, even for statsmodels models.

In [43]:
hp_model = sklearn_sm(sm.OLS,
                      MS(['horsepower']))
X, Y = Auto.drop(columns=['mpg']), Auto['mpg']
cv_results = cross_validate(hp_model,
                            X,
                            Y,
                            cv=Auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.231513517929226

The cross_validate() function in sklearn needs a model (with fit(), predict(), and score() methods), feature data X, and response Y. If we set the cv argument to a number K, it does *K-fold cross-validation. If we set it equal to the total number of observations, it performs **leave-one-out cross-validation (LOOCV). This function returns a dictionary, but we only need the test score (MSE), which in one case is 24.23. We can repeat this for more complex models. To do this automatically, we use a for loop that fits **polynomial models from degree 1 to 5*, calculates the cross-validation error, and saves each result in a list called cv_error. The variable d in the loop is the polynomial degree. Running this might take a few seconds.

In [50]:
cv_error = np.zeros(5)
H = np.array(Auto['horsepower'])
M = sklearn_sm(sm.OLS)
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=Auto.shape[0])
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.23151352, 19.24821312, 19.33498406, 19.42443033, 19.03324353])

outer() method of the np.power() function. The outer() method is applied to an operation that has two arguements (add(), min(), or power()). it has two arrays as arguements and then forms a larger array where the operation is applied to each pair of elements of the two arrays.

In [54]:
A = np.array([3, 5, 9])
B = np.array([2, 4])
np.add.outer(A, B)

array([[ 5,  7],
       [ 7,  9],
       [11, 13]])

In the CV example above, we used K = n, but of course we can also use K < n. The code in very similar to the above(and is significantly faster). Here we use KFold() to partition the data into K = 10 random_state to set a random seed and initialize a vector cv_error in which we will store the CV errors corresponding to the polynomial fits of degrees one to five.

In [59]:
cv_error = np.zeros(5)
cv = KFold(n_splits=10,
           shuffle=True,
           random_state=0) # use same splits for each degree
for i, d in enumerate(range(1,6)):
    X = np.power.outer(H, np.arange(d+1))
    M_CV = cross_validate(M,
                          X,
                          Y,
                          cv=cv)
    cv_error[i] = np.mean(M_CV['test_score'])
cv_error

array([24.20766449, 19.18533142, 19.27626666, 19.47848399, 19.13719696])

LOOCV (Leave-One-Out CV) should be faster for linear models because it has a shortcut formula, but cross_validate() doesn't use that formula—so it's slower in practice.

We also see that using cubic or higher-degree polynomials doesn't improve test error much compared to a simple quadratic model.

In [64]:
validation = ShuffleSplit(n_splits=1,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation);
results['test_score']

array([23.61661707])

The cross_validate() function is flexible—you can use it with different data splitting methods like ShuffleSplit() to easily try the validation set approach instead of K-fold.


In [68]:
validation = ShuffleSplit(n_splits=10,
                          test_size=196,
                          random_state=0)
results = cross_validate(hp_model,
                         Auto.drop(['mpg'], axis=1),
                         Auto['mpg'],
                         cv=validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034164, 1.421845094109187)

One can estimate the variability in the test error by running this. this standard deviation is not a valid estimate of the sampling vairability of the mean test score or the individual scores, since the randomly- selected training samples overlap and hence introduce correlations. but it does give an idea of the monte carlo variation incurred by picking different random folds.

### Bootstrap

The bootstrap is a very useful technique in statistics for estimating the accuracy of a statistic, such as its standard error or variance. One big advantage of the bootstrap is that it works in almost any situation and doesn't require complex mathematical formulas. Instead, it relies on resampling from the data to assess how a statistic might vary if we were to repeat the experiment.

### Estimating the Accuracy of a satistic of Interest

To perform the bootstrap, we repeatedly draw random samples with replacement from our dataset. For each resample, we calculate the statistic of interest. After many such resamples, we look at the variation in those statistics—this gives us an estimate of the standard error.

In Python, even though there are built-in functions for the bootstrap, it's simple enough that we can write our own. For example, suppose we want to estimate the variability of a parameter (like α) based on a dataset that has columns X and Y. We can create a function called alpha_func() that takes a DataFrame and a set of row indices, and returns the estimated value of α using only the selected rows.

This custom function can then be used repeatedly in a bootstrap loop to compute α on many resampled datasets, allowing us to estimate how much α might vary across different samples. This illustrates how flexible and practical the bootstrap method is in real-world data analysis.

In [80]:
Portfolio = load_data('Portfolio')
def alpha_func(D, idx):
   cov_ = np.cov(D[['X','Y']].loc[idx], rowvar=False)
   return ((cov_[1,1] - cov_[0,1]) /
           (cov_[0,0]+cov_[1,1]-2*cov_[0,1]))

This fuction returns an estimate for alpha base on applying the minimum variance formula to the observations indexed by the arguement `idx`. It simply estimates aplha using all 100 observations.

In [84]:
alpha_func(Portfolio, range(100))

0.5758320745928298

Next we randomly select 100 obervations from range(100), with replacement. Constructing a new bootstrap data set and recomputing aplha head based on the new data set.

In [89]:
rng = np.random.default_rng(0)
alpha_func(Portfolio,
           rng.choice(100,
                      100,
                      replace=True))

0.6074452469619002

This process can be generalized to create a simple function boot_SE() for computing the bootstrap standard error for arbitrary functions that take only a data frame as an arguement.

In [94]:
def boot_SE(func,
            D,
            n=None,
            B=1000,
            seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index,
                         n,
                         replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

The use of `_` as a loop variable in `for _ in range(8)`. This is often used if the value of the counter is unimportand and simply makes sure the loop is executed `B` times.

In [99]:
alpha_SE = boot_SE(alpha_func,
                   Portfolio,
                   B=1000,
                   seed=0)
alpha_SE

0.09118176521277577

lets use our function to evaluate the accuracy of our estimate of alpha using B = 1,000 bootstrap replications.

The final output shows that the bootstrap estimate for SE(alpha head) is 0.0912

### Estimating the Accuracy of a Linear Regression Model

The bootstrap can be used to measure how much the coefficients and predictions from a model might vary. In this example, we apply it to a linear regression model that predicts mpg from horsepower using the Auto dataset. We'll check the variability of the intercept and slope estimates and compare the bootstrap results to those from standard formulas.

To do this, we use a custom boot_SE() function, which needs a function that takes a DataFrame D and row indices idx. Since we're bootstrapping a specific regression model, we write a general-purpose function boot_OLS() that takes a model formula and uses the data to refit the model on each bootstrap sample.

We also use the clone() function to ensure any derived features (like from poly()) are recomputed for each resample. This helps us get more accurate estimates of model variability using the bootstrap.

In [109]:
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

This is not quite what is needed as the first argument to boot_SE(). The first two arguments which specify the model will not change in the bootstrap process, and we wouuld like to freeze them. The function `partial()` from `functools` module does precisely this: it takes a function as an arguement, and freezes some of its arguements, strating from the left. We use it to freeze the first two model-formula arguments of boot_OLS()

In [112]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

hp_func will show that it has two arguments D and idx it is a version of boot_OLS() with the first two arguments frozen and hence is ideal as the first arugment for boot_SE().

In [115]:
rng = np.random.default_rng(0)
np.array([hp_func(Auto,
          rng.choice(Auto.index,
                     392,
                     replace=True)) for _ in range(10)])

array([[39.12226577, -0.1555926 ],
       [37.18648613, -0.13915813],
       [37.46989244, -0.14112749],
       [38.56723252, -0.14830116],
       [38.95495707, -0.15315141],
       [39.12563927, -0.15261044],
       [38.45763251, -0.14767251],
       [38.43372587, -0.15019447],
       [37.87581142, -0.1409544 ],
       [37.95949036, -0.1451333 ]])

hp_func() function can now be used in order to create bootstrap estimates for the intercept and slope terms by randomly sampling from among the observations with replacement. We first demostrate its utility on 10 bootstrap samples.

In [123]:
hp_se = boot_SE(hp_func,
                Auto,
                B=1000,
                seed=10)
hp_se

intercept     0.731176
horsepower    0.006092
dtype: float64

We use the boot_SE() function to compute the standard errors of 1000 bootstrap estimates for the intercept and slope terms.

The bootstrap estimates the intercept as 0.85 and the slope as 0.0074. These values show the typical variation we might expect if we repeated the sampling. As mentioned earlier, we can also calculate standard errors for regression coefficients using standard formulas. These can be easily obtained using the summarize() function from ISLP.sm.

In [126]:
hp_model.fit(Auto, Auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

The standard error estimates from formulas are 0.717 for the intercept and 0.006 for the slope. These are a bit different from the bootstrap results (0.85 and 0.0074), but that’s not a problem—in fact, it highlights a key strength of the bootstrap.

Standard formulas assume certain things, like a correct linear model and fixed predictor values. They also estimate noise (σ²) using residuals, which can be inflated if the true relationship is non-linear, as seen in the data.

The bootstrap, however, does not rely on these assumptions, so it's often more realistic and accurate, especially when the model is misspecified.

When we fit a quadratic model (which better fits the data), the bootstrap and formula-based estimates for standard errors match more closely, showing improved agreement when the model is more appropriate.

In [133]:
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS,
                    quad_model,
                    'mpg')
boot_SE(quad_func, Auto, B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

We now compute the bootstrap standard errors and compare them with the standard regression estimates after fitting a quadratic model to the data. Since this model fits the data better the bootstrap and formual- based estimates for the intercept, slope and quadratic term are now more similar. This shows that when the model fits well both methods give consistent results.

In [139]:
M = sm.OLS(Auto['mpg'],
           quad_model.fit_transform(Auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64

We compare the results to the standard errors computed using sm.OLS().